In [2]:
from glob import glob
import shutil
import cv2
from pathlib import Path
from tqdm.auto import tqdm
import json
import numpy as np

src_dir, dst_dir  = 'my_mobile_1fps', 'mobile_enlarge'
def enlarge(src_dir, dst_dir):
    src_jpgs = glob(f'{src_dir}/*.jpg')
    shutil.rmtree(dst_dir)
    Path(dst_dir).mkdir()
    NX, NY = 2400, 1350
    ox, oy  = (NX-1920)//2, (NY-1080)//2
    
    for src in tqdm(src_jpgs):
        im = cv2.imread(src)
        im = cv2.resize(im, (NX, NY))
        im = im[oy:oy+1080, ox:ox+1920]
        cv2.imwrite(f'{dst_dir}/{Path(src).name}', im, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
    
    src_jsons = glob(f'{src_dir}/*.json')
    r = NX/1920
    for src in tqdm(src_jsons):
        with open(src) as f:
            anno = json.load(f)
        objs = anno['shapes']
        for obj in objs:
            pts = np.array(obj['points']) * r
            pts = pts  - np.array([ox, oy])
            obj['points'] = pts.tolist()
        with open(f'{dst_dir}/{Path(src).name}', 'w') as f:
            json.dump(anno, f)
        

In [ ]:
for src_dir in tqdm(glob('20201*')):
    dst_dir = f'enlarge_{src_dir}'
    shutil.rmtree(dst_dir, ignore_errors=True)
    Path(dst_dir).mkdir()
    enlarge(src_dir, dst_dir)
    

In [12]:
annos = glob('mobile_enlarge/*.json')
anno = annos[0]
for anno in tqdm(annos):
    dst = 'enlarge_' + '_'.join(Path(anno).stem.split('_')[1:3])
    shutil.copy(anno, dst)

In [50]:
for v in tqdm(sorted(glob('enlarge_20201*'))):
    jpgs = sorted(glob(f'{v}/*.jpg'))[::15]  
    for jpg in jpgs: 
        json_path = Path(jpg).with_suffix('.json')
        if json_path.exists():
            base = '_'.join(jpg.split('_')[:-1])
            start = int(jpg[-7:-4])
            for i in range(1, 15):
                no = start + i
                dst = f'{base}_{no:03d}.json'
                shutil.copy(json_path, dst)
                with open(dst) as f:
                    anno = json.load(f)
                anno['imagePath'] = Path(dst).stem + '.jpg'
                with open(dst, 'w') as f:
                    json.dump(anno, f)
                

In [44]:
anno

{'version': '4.5.6',
 'flags': {},
 'shapes': [{'label': 'swoon',
   'points': [[1573.814186030983, 326.51620370370375],
    [1589.9963274572651, 326.51620370370375],
    [1593.4289029113245, 316.3888888888889],
    [1590.4866953792734, 265.7523148148147],
    [1575.2852897970083, 235.85262345679013],
    [1554.1994691506409, 227.17206790123453],
    [1516.441139155983, 267.19907407407413],
    [1520.8544504540596, 277.80864197530866]],
   'group_id': None,
   'shape_type': 'polygon',
   'flags': {}}],
 'imagePath': 'mobile_20201129_172148_285.jpg',
 'imageData': None,
 'imageHeight': 1080,
 'imageWidth': 1920}

In [48]:
annok

{'version': '4.5.6',
 'flags': {},
 'shapes': [{'label': 'swoon',
   'points': [[1573.814186030983, 326.51620370370375],
    [1589.9963274572651, 326.51620370370375],
    [1593.4289029113245, 316.3888888888889],
    [1590.4866953792734, 265.7523148148147],
    [1575.2852897970083, 235.85262345679013],
    [1554.1994691506409, 227.17206790123453],
    [1516.441139155983, 267.19907407407413],
    [1520.8544504540596, 277.80864197530866]],
   'group_id': None,
   'shape_type': 'polygon',
   'flags': {}}],
 'imagePath': 'mobile_20201129_172148_286.jpg',
 'imageData': None,
 'imageHeight': 1080,
 'imageWidth': 1920}